In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from xgboost import XGBRFRegressor
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

from sklearn.utils.class_weight import compute_sample_weight

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline as imb_make_pipeline

/Users/clarissarjtai/miniforge3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [287]:
df_with_distance = pd.read_csv("preprocessed_data/df_insp_with_distance-v2.csv.zip")

/var/folders/_y/qzymmsdj5jj6nb7_yhb63h0r0000gn/T/ipykernel_67915/241097410.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_with_distance = pd.read_csv("preprocessed_data/df_insp_with_distance-v2.csv.zip")


In [290]:
print(df_with_distance.shape)
df_with_distance.head(10)

(148693, 33)


,Unnamed: 0,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,...,Census Tract,BIN,BBL,NTA,ClosestWorkingTankDis,ClosestWorkingTankDate,ClosestWorkingTankConfirmNum,ClosestBadTankDis,ClosestBadTankDate,ClosestBadTankConfirmNum
0,0,41678706,TITOS EMPANADAS Y MAS,Bronx,1221,JEROME AVENUE,10452.0,9178193056,Latin American,06/03/2019,...,21900.0,2003083.0,2.025060e+09,BX26,0.331346,09/09/2020,WTI7992302276,5.422600,01/01/2021,WTI1718389770
1,1,50065644,NEW GRAND BUFFET,Queens,3445,48TH ST,11101.0,7182788889,Chinese,05/18/2019,...,17100.0,4000822.0,4.001200e+09,QN31,0.425275,08/25/2020,WTI6227302551,0.659718,01/01/2021,WTI1718389770
2,2,41029356,THE DUPLEX,Manhattan,61,CHRISTOPHER STREET,10014.0,2122555438,American,12/17/2019,...,7300.0,1082224.0,1.006100e+09,MN23,0.051459,11/02/2020,WTI8403171362,1.463683,07/28/2015,WTI8645897156
3,3,50058634,LA FLOR DEL CANARIO #2 RESTAURANT PIZZERIA,Brooklyn,2225,PITKIN AVENUE,11207.0,7184855000,Spanish,11/21/2017,...,115200.0,3088794.0,3.039940e+09,BK82,0.849993,10/14/2020,WTI0264803420,6.166759,01/01/2021,WTI1718389770
4,4,50044708,21 BAR,Manhattan,4318,BROADWAY,10033.0,9179752690,American,06/13/2018,...,27100.0,1063931.0,1.021640e+09,MN35,0.265592,10/28/2020,WTI1137511076,6.500742,01/01/2021,WTI1718389770
5,5,41549759,CHIPOTLE MEXICAN GRILL,Manhattan,235,WEST 56 STREET,10019.0,6466266520,Tex-Mex,02/13/2017,...,13700.0,1070014.0,1.010288e+09,MN17,0.000452,10/06/2020,WTI8515487754,0.963038,05/08/2015,WTI5981923701
6,6,41426561,BROADWAY BAKERY,Queens,8109,41 AVENUE,11373.0,3472578759,Bakery Products/Desserts,07/05/2018,...,26901.0,4307358.0,4.014928e+09,QN29,0.302896,08/20/2020,WTI4449194756,2.311711,01/01/2021,WTI1718389770
7,7,50106966,SUSHI MARKET,Queens,24825,NORTHERN BLVD,11362.0,3474084170,Japanese,08/10/2021,...,147900.0,4169059.0,4.081180e+09,QN45,2.351889,01/27/2020,WTI3755708068,9.814211,01/01/2021,WTI1718389770
8,8,41112470,VILLA MIA PIZZA,Brooklyn,526,3 AVENUE,11215.0,7187883330,Pizza,11/21/2019,...,11700.0,3023074.0,3.010260e+09,BK37,0.542898,06/25/2020,WTI7811492225,3.222641,07/28/2015,WTI8645897156
9,9,41611664,SERVICE BAR 6,Queens,110-00,ROCKAWAY BOULEVARD,11420.0,7182153311,American,12/07/2016,...,86400.0,4457718.0,4.115430e+09,QN55,1.674951,05/30/2020,WTI9603835434,7.511339,01/01/2021,WTI1718389770


In [386]:
df = df_with_distance.copy()
# Convert A, B, C to 3, 2, 1 for encodings, drop all other ratings
grades = ["A", "B", "C"]
df = df[df['GRADE'].isin(grades)]
df['SCORE'] = df.loc[:, 'GRADE']
df.loc[df["SCORE"] == "A", "GRADE"] = 3
df.loc[df["SCORE"] == "B", "GRADE"] = 2
df.loc[df["SCORE"] == "C", "GRADE"] = 1

# Replace violation code, zipcode and cuisine type with its average grade:
target = df["GRADE"]
te = TargetEncoder(cols=["VIOLATION CODE", "ZIPCODE", "CUISINE DESCRIPTION"])
te.fit(df, target)
df = te.transform(df)

# Peform OHE on Boro and Critical Flag:
df = pd.get_dummies(df, columns=["BORO"])
df = pd.get_dummies(df, columns=["CRITICAL FLAG"])

print(df.columns)
df = df.drop(["DBA", 
              "BUILDING", 
              "STREET",
              "PHONE",
              "ACTION",
              "VIOLATION DESCRIPTION",
              "Latitude",
              "Longitude",
              "Community Board",
              "Council District",
              "Census Tract", 
              "RECORD DATE",
              "INSPECTION DATE",
              "INSPECTION TYPE",
              "CRITICAL FLAG_Critical",
              "BIN",
              "SCORE",
              "NTA",
              'Unnamed: 0'], axis=1)
df = df.dropna()
df.head()

Index(['Unnamed: 0', 'CAMIS', 'DBA', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'SCORE', 'GRADE', 'GRADE DATE', 'RECORD DATE',
       'INSPECTION TYPE', 'Latitude', 'Longitude', 'Community Board',
       'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA',
       'ClosestWorkingTankDis', 'ClosestWorkingTankDate',
       'ClosestWorkingTankConfirmNum', 'ClosestBadTankDis',
       'ClosestBadTankDate', 'ClosestBadTankConfirmNum', 'BORO_Bronx',
       'BORO_Brooklyn', 'BORO_Manhattan', 'BORO_Queens', 'BORO_Staten Island',
       'CRITICAL FLAG_Critical', 'CRITICAL FLAG_Not Critical'],
      dtype='object')


,CAMIS,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,GRADE DATE,BBL,ClosestWorkingTankDis,ClosestWorkingTankDate,ClosestWorkingTankConfirmNum,ClosestBadTankDis,ClosestBadTankDate,ClosestBadTankConfirmNum,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical
0,41678706,2.622771,2.675366,2.597904,2,06/03/2019,2.025060e+09,0.331346,09/09/2020,WTI7992302276,5.422600,01/01/2021,WTI1718389770,1,0,0,0,0,0
1,50065644,2.797419,2.714226,2.665241,2,05/18/2019,4.001200e+09,0.425275,08/25/2020,WTI6227302551,0.659718,01/01/2021,WTI1718389770,0,0,0,1,0,0
2,41029356,2.784406,2.834724,2.660336,3,12/17/2019,1.006100e+09,0.051459,11/02/2020,WTI8403171362,1.463683,07/28/2015,WTI8645897156,0,0,1,0,0,1
3,50058634,2.653798,2.609433,2.807527,3,11/21/2017,3.039940e+09,0.849993,10/14/2020,WTI0264803420,6.166759,01/01/2021,WTI1718389770,0,1,0,0,0,0
4,50044708,2.607099,2.834724,2.785281,3,06/13/2018,1.021640e+09,0.265592,10/28/2020,WTI1137511076,6.500742,01/01/2021,WTI1718389770,0,0,1,0,0,0


In [387]:
df.columns.values

array(['CAMIS', 'ZIPCODE', 'CUISINE DESCRIPTION', 'VIOLATION CODE',
       'GRADE', 'GRADE DATE', 'BBL', 'ClosestWorkingTankDis',
       'ClosestWorkingTankDate', 'ClosestWorkingTankConfirmNum',
       'ClosestBadTankDis', 'ClosestBadTankDate',
       'ClosestBadTankConfirmNum', 'BORO_Bronx', 'BORO_Brooklyn',
       'BORO_Manhattan', 'BORO_Queens', 'BORO_Staten Island',
       'CRITICAL FLAG_Not Critical'], dtype=object)

In [310]:
# check whether CAMIS in unique ID
print("Number of restaurant: ", pd.unique(df['CAMIS']).shape)
print("Dataframe shape: ", df.shape)

Number of restaurant:  (22527,)
Dataframe shape:  (144130, 18)


In [388]:
# transform to datetime object
def transform_to_datetime(df, colname:str, format_str:str = '%m/%d/%Y'):
    import datetime as dt 
    return [dt.datetime.strptime(str(date_string), format_str) for date_string in df[colname].values ]
df['GRADE DATE'] = transform_to_datetime(df, 'GRADE DATE')
df['ClosestWorkingTankDate'] = transform_to_datetime(df, 'ClosestWorkingTankDate')
df['ClosestBadTankDate'] = transform_to_datetime(df, 'ClosestBadTankDate')

# Find records after 2019
df.drop_duplicates(inplace=True)
print("Number of records after 2019: ", sum(df['GRADE DATE'] > dt.datetime.strptime("01/01/2019", '%m/%d/%Y')))
df.reset_index(drop = True, inplace= True)
df['GRADE'] = df['GRADE'].astype(float)

Number of records after 2019:  87939


In [389]:
print("Most recent grade date: ", df['GRADE DATE'].max())
print("Earliest grade date: ", df['GRADE DATE'].min())
print("Most recent ClosestWorkingTankDate: ", df['ClosestWorkingTankDate'].max())
print("Earliest ClosestWorkingTankDate: ", df['ClosestWorkingTankDate'].min())
print("Most recent ClosestBadTankDate: ", df['ClosestBadTankDate'].max())
print("Earliest ClosestBadTankDate: ", df['ClosestBadTankDate'].min())

Most recent grade date:  2022-04-08 00:00:00
Earliest grade date:  2015-05-14 00:00:00
Most recent ClosestWorkingTankDate:  2020-12-31 00:00:00
Earliest ClosestWorkingTankDate:  2020-01-07 00:00:00
Most recent ClosestBadTankDate:  2021-01-01 00:00:00
Earliest ClosestBadTankDate:  2015-05-08 00:00:00


# Average inspection by restaurant by date

In [399]:
# Check if BBL code is unique for each restaurant
df[['BBL', 'CAMIS']].drop_duplicates().shape[0] == df.CAMIS.unique().shape[0] # True

True

In [400]:
# Average inspection by restaurant by date
df_date = df[['CAMIS', 'GRADE DATE','ClosestWorkingTankDate', 'ClosestBadTankDate']].drop_duplicates().sort_values(by = ['CAMIS', 'GRADE DATE']).reset_index(drop = True)
df_grouped = df.drop(columns=['ClosestWorkingTankDate', 'ClosestBadTankDate']).groupby(['CAMIS', 'GRADE DATE']).mean().reset_index()

df_grouped = pd.concat([df_grouped, df_date.drop(columns=['CAMIS', 'GRADE DATE'])], axis= 1)
print(df_grouped.shape)
df_grouped.head(-10)

(56449, 17)


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,BBL,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,ClosestWorkingTankDate,ClosestBadTankDate
0,30075445,2019-06-11,2.876777,2.778831,2.847154,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,1.000000,2020-12-14,2021-01-01
1,30075445,2019-11-04,2.876777,2.778831,2.621810,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-12-14,2021-01-01
2,30075445,2021-08-10,2.876777,2.778831,2.785281,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-12-14,2021-01-01
3,30112340,2019-03-04,2.801389,2.913240,2.792569,3.0,3.011970e+09,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.666667,2020-09-14,2015-07-28
4,30112340,2021-08-05,2.801389,2.913240,2.634818,2.0,3.011970e+09,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.333333,2020-09-14,2015-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56434,50119057,2022-03-10,2.826147,2.891645,2.771791,3.0,1.000110e+09,0.024849,0.607054,0.0,0.0,1.0,0.0,0.0,0.500000,2020-06-09,2015-07-28
56435,50119087,2022-03-07,2.831125,2.707921,2.847673,3.0,1.010480e+09,0.055791,1.205010,0.0,0.0,1.0,0.0,0.0,1.000000,2020-07-23,2015-05-08
56436,50119194,2022-03-25,2.805080,2.794547,2.725261,3.0,3.024330e+09,0.569764,2.832640,0.0,1.0,0.0,0.0,0.0,0.000000,2020-08-20,2015-07-28
56437,50119223,2022-03-03,2.720213,2.777778,2.809263,3.0,3.086671e+09,0.351167,9.495950,0.0,1.0,0.0,0.0,0.0,0.666667,2020-03-10,2015-07-28


# Find previous inspection

In [401]:
# Calculate recency: recent visit - previous visit
camis = df_grouped['CAMIS'].unique()
new_df = pd.DataFrame()
for idx in camis:
    # filter the records of specific restuarant using CAMIS
    subset = df_grouped[df_grouped.CAMIS == idx].sort_values(by = 'GRADE DATE')
    if subset.shape[0] == 1:
        continue
    prev_df = pd.DataFrame(index=subset.index.values)
    prev_df['GRADE DATE'] = subset['GRADE DATE']
    # prev_df['PREV_CRITICAL FLAG_Not Applicable'] = subset.shift(1)['CRITICAL FLAG_Not Applicable']
    # prev_df['PREV_CRITICAL FLAG_Not Critical'] = subset.shift(1)['CRITICAL FLAG_Not Critical']
    # find previous grading
    
    # find previous grading date and RECENCY
    df_date = subset[['GRADE', 'GRADE DATE']].drop_duplicates()
    if df_date.shape[0] != df_date['GRADE DATE'].unique().shape[0]:
        print(idx)
        break
    df_date['PREV_GRADE'] = df_date.shift(1)['GRADE']
    df_date['PREV_GRADE DATE'] = df_date.shift(1)['GRADE DATE']
    df_date['RECENCY'] = df_date['GRADE DATE'] - df_date.shift(1)['GRADE DATE']
    prev_df = prev_df.merge(df_date.drop(columns='GRADE'), how = 'left', on='GRADE DATE').set_index(subset.index.values)
    # append to new dataframe
    new_df = pd.concat([new_df, prev_df], axis=0)

In [402]:
# Filter rows after year 2019
df_concat = pd.concat([df_grouped, new_df.drop(columns=['GRADE DATE'])], axis=1)
# filter rows with date
df_concat_filtered = df_concat[df_grouped['GRADE DATE'] > dt.datetime.strptime("01/01/2019", '%m/%d/%Y')]
print(df_concat_filtered.shape)
df_concat_filtered.head(10)

(36880, 20)


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,BBL,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,ClosestWorkingTankDate,ClosestBadTankDate,PREV_GRADE,PREV_GRADE DATE,RECENCY
0,30075445,2019-06-11,2.876777,2.778831,2.847154,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,1.000000,2020-12-14,2021-01-01,NaN,NaT,NaT
1,30075445,2019-11-04,2.876777,2.778831,2.621810,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-12-14,2021-01-01,3.0,2019-06-11,146 days
2,30075445,2021-08-10,2.876777,2.778831,2.785281,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-12-14,2021-01-01,3.0,2019-11-04,645 days
3,30112340,2019-03-04,2.801389,2.913240,2.792569,3.0,3.011970e+09,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.666667,2020-09-14,2015-07-28,NaN,NaT,NaT
4,30112340,2021-08-05,2.801389,2.913240,2.634818,2.0,3.011970e+09,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.333333,2020-09-14,2015-07-28,3.0,2019-03-04,885 days
5,30191841,2019-06-06,2.831125,2.814634,2.857626,3.0,1.010480e+09,0.004614,1.120030,0.0,0.0,1.0,0.0,0.0,0.500000,2020-12-15,2015-05-08,NaN,NaT,NaT
6,30191841,2022-01-04,2.831125,2.814634,2.870896,3.0,1.010480e+09,0.004614,1.120030,0.0,0.0,1.0,0.0,0.0,0.666667,2020-12-15,2015-05-08,3.0,2019-06-06,943 days
7,40356018,2019-11-08,2.868895,2.834724,2.855589,3.0,3.069970e+09,0.315359,9.367778,0.0,1.0,0.0,0.0,0.0,0.666667,2020-11-04,2015-07-28,NaN,NaT,NaT
8,40356018,2022-02-01,2.868895,2.834724,2.622308,3.0,3.069970e+09,0.315359,9.367778,0.0,1.0,0.0,0.0,0.0,0.000000,2020-11-04,2015-07-28,3.0,2019-11-08,816 days
9,40356483,2019-05-03,2.794118,2.770759,2.854308,3.0,3.084310e+09,2.320265,8.494743,0.0,1.0,0.0,0.0,0.0,0.666667,2020-03-11,2015-07-28,NaN,NaT,NaT


In [403]:
# Null values in each column
df_concat_filtered.isnull().sum()

CAMIS                             0
GRADE DATE                        0
ZIPCODE                           0
CUISINE DESCRIPTION               0
VIOLATION CODE                    0
GRADE                             0
BBL                               0
ClosestWorkingTankDis             0
ClosestBadTankDis                 0
BORO_Bronx                        0
BORO_Brooklyn                     0
BORO_Manhattan                    0
BORO_Queens                       0
BORO_Staten Island                0
CRITICAL FLAG_Not Critical        0
ClosestWorkingTankDate            0
ClosestBadTankDate                0
PREV_GRADE                    12556
PREV_GRADE DATE               12556
RECENCY                       12556
dtype: int64

# Frequency of inspection for each restaurant after 2019

In [404]:
# Calculate frequency: time of grading within three year
df_freq = pd.DataFrame(df_concat_filtered.groupby(['CAMIS']).size(), columns=["frequency"])
print(df_freq.frequency.unique())
print(df_freq.head(10))
# Inspection is not once a year
df_concat_filtered[df_concat_filtered.CAMIS == 41319107]

[3 2 1 4 5]
          frequency
CAMIS              
30075445          3
30112340          2
30191841          2
40356018          2
40356483          2
40356731          2
40357217          2
40359480          1
40359705          2
40360045          2


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,BBL,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,ClosestWorkingTankDate,ClosestBadTankDate,PREV_GRADE,PREV_GRADE DATE,RECENCY
9885,41319107,2019-01-28,2.670737,2.834724,2.731677,3.0,4.049760e+09,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.666667,2020-10-26,2021-01-01,NaN,NaT,NaT
9886,41319107,2019-05-30,2.670737,2.834724,2.709183,3.0,4.049760e+09,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.000000,2020-10-26,2021-01-01,3.0,2019-01-28,122 days
9887,41319107,2019-09-23,2.670737,2.834724,2.784434,3.0,4.049760e+09,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.666667,2020-10-26,2021-01-01,3.0,2019-05-30,116 days
9888,41319107,2020-01-30,2.670737,2.834724,2.646220,3.0,4.049760e+09,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.500000,2020-10-26,2021-01-01,3.0,2019-09-23,129 days
9889,41319107,2022-01-27,2.670737,2.834724,2.729957,3.0,4.049760e+09,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.666667,2020-10-26,2021-01-01,3.0,2020-01-30,728 days


# Add BBL rodent inspection

In [372]:
df_rodent = pd.read_csv("preprocessed_data/Rodent_Inspection_Cleaned.csv")
print("Rodent_Inspection_Cleaned.csv has {0} rows and {1} columns.".format(df_rodent.shape[0],df_rodent.shape[1]))
print(df_rodent.columns)

Rodent_Inspection_Cleaned.csv has 2044392 rows and 18 columns.
Index(['Unnamed: 0', 'INSPECTION_TYPE', 'JOB_TICKET_OR_WORK_ORDER_ID',
       'JOB_ID', 'JOB_PROGRESS', 'BBL', 'BORO_CODE', 'BLOCK', 'LOT',
       'HOUSE_NUMBER', 'STREET_NAME', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE',
       'BOROUGH', 'INSPECTION_DATE', 'RESULT', 'APPROVED_DATE'],
      dtype='object')


In [373]:
meta_rodent = df_rodent.drop(columns=['BBL','INSPECTION_DATE', 'RESULT', 'Unnamed: 0'])
df_rodent = df_rodent[['BBL','INSPECTION_DATE', 'RESULT']]
df_rodent.head()

,BBL,INSPECTION_DATE,RESULT
0,3048960003,2022-02-04 12:00:06,Rat Activity
1,2051410120,2017-07-25 15:25:08,Passed
2,1004000057,2019-08-12 13:50:23,Rat Activity
3,1002930019,2018-09-25 14:02:03,Passed
4,3012380033,2022-02-11 15:47:00,Passed


In [374]:
# 1: failed in some way
df_rodent['RESULT'] = np.where(df_rodent['RESULT'] == 'Rat Activity', 1, df_rodent['RESULT'])
df_rodent['RESULT'] = np.where(df_rodent['RESULT'] == 'Failed for Other R', 1, df_rodent['RESULT'])
df_rodent['RESULT'] = np.where(df_rodent['RESULT'] != 1, 0, df_rodent['RESULT'])
df_rodent.head()

,BBL,INSPECTION_DATE,RESULT
0,3048960003,2022-02-04 12:00:06,1
1,2051410120,2017-07-25 15:25:08,0
2,1004000057,2019-08-12 13:50:23,1
3,1002930019,2018-09-25 14:02:03,0
4,3012380033,2022-02-11 15:47:00,0


# get the most recent inspection for each BBL

In [385]:
df_rodent_BBL = df_rodent.groupby(by = ['BBL']).last().reset_index()
df_rodent_BBL.head(10)

,BBL,INSPECTION_DATE,RESULT
0,1000000000,2020-03-05 10:35:00,1
1,1000010101,2009-12-07 15:52:59,0
2,1000020001,2011-02-18 16:01:14,0
3,1000020023,2011-02-18 15:53:29,0
4,1000030001,2018-07-26 12:04:45,1
5,1000030002,2016-10-28 14:11:25,1
6,1000030003,2014-02-24 13:15:56,1
7,1000030004,2018-06-25 12:05:32,0
8,1000030010,2016-10-28 14:31:08,0
9,1000047501,2016-10-28 13:26:10,0


# Merge Rodent Inspection to restaurant and watertank data on BBL

In [405]:
print(df_rodent_BBL.shape,df_concat_filtered.shape)

(296456, 3) (36880, 20)


In [419]:
df_final = df_concat_filtered.merge(df_rodent_BBL, how='left', on='BBL')
print(df_final.shape)
df_final.head() 

(36880, 22)


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,BBL,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,...,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,ClosestWorkingTankDate,ClosestBadTankDate,PREV_GRADE,PREV_GRADE DATE,RECENCY,INSPECTION_DATE,RESULT
0,30075445,2019-06-11,2.876777,2.778831,2.847154,3.0,2.041270e+09,0.484806,7.207183,1.0,...,0.0,0.0,1.000000,2020-12-14,2021-01-01,NaN,NaT,NaT,NaN,NaN
1,30075445,2019-11-04,2.876777,2.778831,2.621810,3.0,2.041270e+09,0.484806,7.207183,1.0,...,0.0,0.0,0.000000,2020-12-14,2021-01-01,3.0,2019-06-11,146 days,NaN,NaN
2,30075445,2021-08-10,2.876777,2.778831,2.785281,3.0,2.041270e+09,0.484806,7.207183,1.0,...,0.0,0.0,0.000000,2020-12-14,2021-01-01,3.0,2019-11-04,645 days,NaN,NaN
3,30112340,2019-03-04,2.801389,2.913240,2.792569,3.0,3.011970e+09,0.226460,4.392523,0.0,...,0.0,0.0,0.666667,2020-09-14,2015-07-28,NaN,NaT,NaT,2016-09-19 15:25:51,0.0
4,30112340,2021-08-05,2.801389,2.913240,2.634818,2.0,3.011970e+09,0.226460,4.392523,0.0,...,0.0,0.0,0.333333,2020-09-14,2015-07-28,3.0,2019-03-04,885 days,2016-09-19 15:25:51,0.0


In [420]:
# Null values in each column
df_final.isnull().sum()

CAMIS                             0
GRADE DATE                        0
ZIPCODE                           0
CUISINE DESCRIPTION               0
VIOLATION CODE                    0
GRADE                             0
BBL                               0
ClosestWorkingTankDis             0
ClosestBadTankDis                 0
BORO_Bronx                        0
BORO_Brooklyn                     0
BORO_Manhattan                    0
BORO_Queens                       0
BORO_Staten Island                0
CRITICAL FLAG_Not Critical        0
ClosestWorkingTankDate            0
ClosestBadTankDate                0
PREV_GRADE                    12556
PREV_GRADE DATE               12556
RECENCY                       12556
INSPECTION_DATE                7832
RESULT                         7832
dtype: int64

In [422]:
# fill na values # For date columns leave it as it is, the others fill with zeros
df_final.RECENCY = df_final.RECENCY.dt.days.fillna(0)
df_final.PREV_GRADE = df_final.PREV_GRADE.fillna(0)
df_final.RESULT = df_final.RESULT.fillna(0)

In [426]:
# Null values in each column
print(df_final.columns)
display(df_final.head())
df_final.isnull().sum()

Index(['CAMIS', 'GRADE DATE', 'ZIPCODE', 'CUISINE DESCRIPTION',
       'VIOLATION CODE', 'GRADE', 'BBL', 'ClosestWorkingTankDis',
       'ClosestBadTankDis', 'BORO_Bronx', 'BORO_Brooklyn', 'BORO_Manhattan',
       'BORO_Queens', 'BORO_Staten Island', 'CRITICAL FLAG_Not Critical',
       'ClosestWorkingTankDate', 'ClosestBadTankDate', 'PREV_GRADE',
       'PREV_GRADE DATE', 'RECENCY', 'INSPECTION_DATE', 'RESULT'],
      dtype='object')


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,BBL,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,...,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,ClosestWorkingTankDate,ClosestBadTankDate,PREV_GRADE,PREV_GRADE DATE,RECENCY,INSPECTION_DATE,RESULT
0,30075445,2019-06-11,2.876777,2.778831,2.847154,3.0,2.041270e+09,0.484806,7.207183,1.0,...,0.0,0.0,1.000000,2020-12-14,2021-01-01,0.0,NaT,0.0,NaN,0.0
1,30075445,2019-11-04,2.876777,2.778831,2.621810,3.0,2.041270e+09,0.484806,7.207183,1.0,...,0.0,0.0,0.000000,2020-12-14,2021-01-01,3.0,2019-06-11,146.0,NaN,0.0
2,30075445,2021-08-10,2.876777,2.778831,2.785281,3.0,2.041270e+09,0.484806,7.207183,1.0,...,0.0,0.0,0.000000,2020-12-14,2021-01-01,3.0,2019-11-04,645.0,NaN,0.0
3,30112340,2019-03-04,2.801389,2.913240,2.792569,3.0,3.011970e+09,0.226460,4.392523,0.0,...,0.0,0.0,0.666667,2020-09-14,2015-07-28,0.0,NaT,0.0,2016-09-19 15:25:51,0.0
4,30112340,2021-08-05,2.801389,2.913240,2.634818,2.0,3.011970e+09,0.226460,4.392523,0.0,...,0.0,0.0,0.333333,2020-09-14,2015-07-28,3.0,2019-03-04,885.0,2016-09-19 15:25:51,0.0


CAMIS                             0
GRADE DATE                        0
ZIPCODE                           0
CUISINE DESCRIPTION               0
VIOLATION CODE                    0
GRADE                             0
BBL                               0
ClosestWorkingTankDis             0
ClosestBadTankDis                 0
BORO_Bronx                        0
BORO_Brooklyn                     0
BORO_Manhattan                    0
BORO_Queens                       0
BORO_Staten Island                0
CRITICAL FLAG_Not Critical        0
ClosestWorkingTankDate            0
ClosestBadTankDate                0
PREV_GRADE                        0
PREV_GRADE DATE               12556
RECENCY                           0
INSPECTION_DATE                7832
RESULT                            0
dtype: int64

In [427]:
# df_final.to_csv("preprocessed_data/RestWaterRodent_data.csv.zip")

In [429]:
# Keep columns not for training as metadata
print(df_final.shape)
meta_final = df_final[['CAMIS', 'GRADE DATE', 'VIOLATION CODE','ClosestWorkingTankDate', 'ClosestBadTankDate', 'PREV_GRADE DATE', 'INSPECTION_DATE']]
# data for model training
data = df_final.drop(columns=['CAMIS', 'GRADE DATE', 'VIOLATION CODE','ClosestWorkingTankDate', 'ClosestBadTankDate', 'PREV_GRADE DATE', 'INSPECTION_DATE'])
data.head()

(36880, 22)


,ZIPCODE,CUISINE DESCRIPTION,GRADE,BBL,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,PREV_GRADE,RECENCY,RESULT
0,2.876777,2.778831,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0
1,2.876777,2.778831,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,3.0,146.0,0.0
2,2.876777,2.778831,3.0,2.041270e+09,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,3.0,645.0,0.0
3,2.801389,2.913240,3.0,3.011970e+09,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0
4,2.801389,2.913240,2.0,3.011970e+09,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.333333,3.0,885.0,0.0
